In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 786.73it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 714.98it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=4, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

In [6]:
from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

In [ ]:
from training.trainer import train_model

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=7, batchSize=16, warmupSteps=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

inner loop training loss is 7.814536094665527
inner loop training loss is 6.339726448059082
inner loop training loss is 8.375788688659668
inner loop training loss is 7.571019649505615
outer loop losses are [tensor(0.4439, grad_fn=<UnbindBackward0>), tensor(0.8937, grad_fn=<UnbindBackward0>), tensor(0.9641, grad_fn=<UnbindBackward0>), tensor(1.3359, grad_fn=<UnbindBackward0>), tensor(1.6016, grad_fn=<UnbindBackward0>), tensor(1.5502, grad_fn=<UnbindBackward0>), tensor(1.8715, grad_fn=<UnbindBackward0>), tensor(1.7682, grad_fn=<UnbindBackward0>), tensor(1.4481, grad_fn=<UnbindBackward0>)] and loss is 11.877238273620605
outer loop losses are [tensor(0.4439, grad_fn=<UnbindBackward0>), tensor(0.8937, grad_fn=<UnbindBackward0>), tensor(0.9641, grad_fn=<UnbindBackward0>), tensor(1.3359, grad_fn=<UnbindBackward0>), tensor(1.6016, grad_fn=<UnbindBackward0>), tensor(1.5502, grad_fn=<UnbindBackward0>), tensor(1.8715, grad_fn=<UnbindBackward0>), tensor(1.7682, grad_fn=<UnbindBackward0>), tensor(1

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 7.290179252624512
inner loop training loss is 6.441580772399902
inner loop training loss is 8.439443588256836
inner loop training loss is 6.5053887367248535
outer loop losses are [tensor(0.6826, grad_fn=<UnbindBackward0>), tensor(0.7970, grad_fn=<UnbindBackward0>), tensor(1.5267, grad_fn=<UnbindBackward0>), tensor(1.6516, grad_fn=<UnbindBackward0>), tensor(1.5395, grad_fn=<UnbindBackward0>), tensor(1.2794, grad_fn=<UnbindBackward0>)] and loss is 7.4768757820129395
outer loop losses are [tensor(0.6826, grad_fn=<UnbindBackward0>), tensor(0.7970, grad_fn=<UnbindBackward0>), tensor(1.5267, grad_fn=<UnbindBackward0>), tensor(1.6516, grad_fn=<UnbindBackward0>), tensor(1.5395, grad_fn=<UnbindBackward0>), tensor(1.2794, grad_fn=<UnbindBackward0>), tensor(1.9092, grad_fn=<UnbindBackward0>), tensor(1.2700, grad_fn=<UnbindBackward0>), tensor(1.1583, grad_fn=<UnbindBackward0>), tensor(1.3126, grad_fn=<UnbindBackward0>), tensor(0.8755, grad_fn=<UnbindBackward0>), tensor(

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

...

inner loop training loss is 4.727623462677002
inner loop training loss is 3.9513285160064697
inner loop training loss is 5.034641742706299
inner loop training loss is 4.400092601776123
outer loop losses are [tensor(0.8744, grad_fn=<UnbindBackward0>), tensor(1.0884, grad_fn=<UnbindBackward0>), tensor(0.6110, grad_fn=<UnbindBackward0>), tensor(1.0514, grad_fn=<UnbindBackward0>), tensor(0.6518, grad_fn=<UnbindBackward0>), tensor(0.6009, grad_fn=<UnbindBackward0>), tensor(0.4159, grad_fn=<UnbindBackward0>), tensor(0.4258, grad_fn=<UnbindBackward0>), tensor(0.5463, grad_fn=<UnbindBackward0>), tensor(1.0509, grad_fn=<UnbindBackward0>), tensor(0.8310, grad_fn=<UnbindBackward0>), tensor(1.0817, grad_fn=<UnbindBackward0>), tensor(1.2848, grad_fn=<UnbindBackward0>), tensor(1.1282, grad_fn=<UnbindBackward0>)] and loss is 11.642616271972656
outer loop losses are [tensor(0.8744, grad_fn=<UnbindBackward0>), tensor(1.0884, grad_fn=<UnbindBackward0>), tensor(0.6110, grad_fn=<UnbindBackward0>), tensor(

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 4.752189636230469
inner loop training loss is 4.828998565673828
inner loop training loss is 4.835038661956787
inner loop training loss is 4.8066606521606445
outer loop losses are [tensor(0.6058, grad_fn=<UnbindBackward0>), tensor(0.7395, grad_fn=<UnbindBackward0>), tensor(0.7044, grad_fn=<UnbindBackward0>), tensor(0.6449, grad_fn=<UnbindBackward0>), tensor(0.6962, grad_fn=<UnbindBackward0>), tensor(0.7284, grad_fn=<UnbindBackward0>)] and loss is 4.119276523590088
outer loop losses are [tensor(0.6058, grad_fn=<UnbindBackward0>), tensor(0.7395, grad_fn=<UnbindBackward0>), tensor(0.7044, grad_fn=<UnbindBackward0>), tensor(0.6449, grad_fn=<UnbindBackward0>), tensor(0.6962, grad_fn=<UnbindBackward0>), tensor(0.7284, grad_fn=<UnbindBackward0>), tensor(0.7081, grad_fn=<UnbindBackward0>), tensor(1.2026, grad_fn=<UnbindBackward0>), tensor(0.7261, grad_fn=<UnbindBackward0>), tensor(0.7082, grad_fn=<UnbindBackward0>), tensor(0.7397, grad_fn=<UnbindBackward0>), tensor(1

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 5.111295700073242
inner loop training loss is 4.604230880737305
